In [1]:
import os
import csv
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re

In [ ]:
gb_dir = '' #Directory with genbank files trimmed to the size of genomic region processed with RODEO. Names are {RODEO QUERY} + '.gbk'
rod_dir = '' #Directory with RODEO output. In this directory one subdirectory for one genomic region (e.g. RIPPER output)

In [ ]:
out = []

for x in os.listdir(rod_dir):
    inf = open(rod_dir + x + '/main_co_occur.csv')
    inf.next()
    csv_file = list(csv.reader(inf))
    inf.close()
    
    ycao_start = 0
    ycao_end = 0
    
    bgc_start = ''
    bgc_end = ''
    
    for row in csv_file:
        if row[0] == row[3]:
            ycao_start, ycao_end = min(int(row[4]), int(row[5])), max(int(row[4]), int(row[5]))
    start_found = False
    
    for row in csv_file:
        
        if not start_found and max(int(row[4]), int(row[5])) >= ycao_start - 8000:
            start_found = True
            bgc_start = row[3]#min(int(row[4]), int(row[5]))
        
        if start_found and min(int(row[4]), int(row[5])) <= ycao_end + 8000:
            bgc_end = row[3]#max(int(row[4]), int(row[5]))
            
    out.append([csv_file[0][0], csv_file[0][2], 
                bgc_start, 
                bgc_end])

out

In [2]:
contig_edges = []

for protein in out:
    gb_record = SeqIO.parse(gb_dir + protein[1] + '.gbk', "genbank", generic_dna)
    for record in gb_record:
        if record.id == protein[1]:

            start = int(protein[2])
            end = int(protein[3])

            if start == 0 and end == len(record):

                contig_edges.append([protein[0], 'both'])

            elif start == 0:

                contig_edges.append([protein[0], 'start'])

            elif end == len(record):

                contig_edges.append([protein[0], 'end'])

outf = open('contig_edge.csv', 'w')

for x in contig_edges:
    outf.write(','.join(x) + '\n')

outf.close()

NameError: name 'out' is not defined

In [ ]:
def table_reader(x):
    
           
    with open(rod_dir + x + '/main_co_occur.csv', 'r') as infile:
        infile.next()
            
        operon_accs = []
                
        prev_end = 0
        prev_strand = ''
        seed = x

        for row in csv.reader(infile):
            start = min(int(row[4]), int(row[5]))
            if (row[6] == prev_strand) and (start - prev_end < 100):
                operon_accs.append(row[3])
                    
            else:
                if seed in operon_accs:
                    return (operon_accs[0], operon_accs[-1])
                    
                operon_accs = [row[3]]
                    
            prev_end = max(int(row[4]), int(row[5]))
            prev_strand = row[6]
        
        if seed in operon_accs:
            return (operon_accs[0], operon_accs[-1])

In [ ]:
def get_operon_coords(operon_borders):
    operon_coords = {}
    prot_id_regexp = re.compile('[A-Z]{2}_[0-9]+\.[0-9]')
    
    
    for x in os.listdir(gb_dir):
        
        prot_id = x.rstrip('.gbk')
        
        start_id = operon_borders[prot_id][0]
        end_id = operon_borders[prot_id][1]
        
        operon_coords[prot_id] = []
        
        gb_record = SeqIO.parse(gb_dir + x, "genbank", generic_dna)
        for record in gb_record:
            for feature in record.features:
                
                if 'protein_id' in feature.qualifiers:
                    
                    if feature.qualifiers['protein_id'][0]  == start_id:
                        operon_coords[prot_id].append(feature.location.start + 1)# genbank is [1:], python is
                           
                    if feature.qualifiers['protein_id'][0]  == end_id:
                       
                        operon_coords[prot_id].append(int(feature.location.end))
                
                elif 'pseudo' in feature.qualifiers:
                    
                    if 'inference' in feature.qualifiers:
                        
                        inference_prot_id_search = prot_id_regexp.search(feature.qualifiers['inference'][0])
                        
                        if inference_prot_id_search is not None:
                            
                            inference_prot_id = inference_prot_id_search.group(0)
                            
                            if inference_prot_id  == start_id:
                                operon_coords[prot_id].append(feature.location.start + 1)# genbank is [1:], python is

                            if inference_prot_id  == end_id:
                                operon_coords[prot_id].append(int(feature.location.end))
                        else:
                            
                            if feature.qualifiers['locus_tag'][0]  == start_id:
                                operon_coords[prot_id].append(feature.location.start + 1)# genbank is [1:], python is

                            if feature.qualifiers['locus_tag'][0] == end_id:
                                operon_coords[prot_id].append(int(feature.location.end))
    return operon_coords

In [ ]:
def get_bg_list(name, gotta_gbks):
    #dirs = ['/home/gyrase/datatank/Bikmetov/ycao_clusters/ripper_out/rodeo/rodout/' + x for x in os.listdir('/home/gyrase/datatank/Bikmetov/ycao_clusters/ripper_out/rodeo/rodout/') if x in ycaos]
    #dirs += ['rodout/' + x for x in os.listdir('rodout/') if x in ycaos]
    
    inlist = []
    with open('domfilter/' + name + '/' +name + '_table' +'.csv', 'r') as infile:
        inlist = [x for x in csv.reader(infile)]


    li = []

    for line in inlist:
        for dir in gotta_gbks:
            #print(dir + ' is processing')
            with open(rod_dir + dir + '/main_co_occur.csv', 'r') as infile:
                for row in csv.reader(infile):            
                    try:
                        if row[7] == line[0]:
                            li.append(row[3])
                    except:
                        pass
    return li

In [ ]:
def make_antismash_great_again(name, types, operon_coords, gotta_gbks):
    
    inf = open('contig_edge.csv', 'r')
    contig_edge_list = [x[0] for x in csv.reader(inf)]
    inf.close
    
        
    bg_list = get_bg_list(name, gotta_gbks)
    #print bg_list
    
    bg = False
    
    contig_edge = False
    
    for x in os.listdir('trimmed_gbk_headers_corrected/'):
        
        with open('trimmed_gbk_headers_corrected/' + x, 'r') as infile:
            with open('trimmed_gbk_headers_as_improved/' + x, 'w') as outfile:
                
                prot_id = x.rstrip('.gbk')
                
                if prot_id in contig_edge_list:
                    contig_edge = True
                else:
                    contig_edge = False
                print prot_id
                cluster_start = operon_coords[prot_id][0]
                cluster_end = operon_coords[prot_id][1]
               
                for line in infile:
                    
                    if line.startswith('                     /protein_id='):
                        if line.split('\"')[-2] in bg_list:
                            bg = True
                    
                    elif line.startswith('                     /transl_table=') and bg:
                        outfile.write('                     /sec_met="Kind: biosynthetic"' + '\n')
                        bg = False
                    
                    outfile.write(line)
                    if line[0:8] == 'FEATURES':
                        outfile.write('     cluster         ' + '..'.join((str(cluster_start), str(cluster_end))) + '\n' + 
                                      '                     /contig_edge=\"' + str(contig_edge) + '\"' + '\n' +
                                      '                     /product=\"' + types[name] + '\"' + '\n')

In [ ]:
#os.mkdir('trimmed_gbk_headers_as')

In [ ]:
operon_borders = {x : table_reader(x) for x in gotta_gbks}

In [ ]:
operon_coords = get_operon_coords(operon_borders)

In [ ]:
#os.mkdir('trimmed_gbk_headers_as_improved/')

In [ ]:
make_antismash_great_again('tomms', {'tomms': 'thiopeptide'}, operon_coords, gotta_gbks)